# NLP - English tutorial

## Loading data

In [2]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [53]:
nltk.download('subjectivity')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package subjectivity to
[nltk_data]     /home/nelth/nltk_data...
[nltk_data]   Package subjectivity is already up-to-date!
[nltk_data] Downloading package punkt to /home/nelth/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/nelth/nltk_data...


True

In [5]:
n_instances = 100
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
len(subj_docs), len(obj_docs)

(100, 100)

> Each document is represented by a tuple (sentence, label). The sentence is tokenized, so it is represented by a list of strings:

In [29]:
subj_docs[0:3]

[(['smart',
   'and',
   'alert',
   ',',
   'thirteen',
   'conversations',
   'about',
   'one',
   'thing',
   'is',
   'a',
   'small',
   'gem',
   '.'],
  'subj'),
 (['color',
   ',',
   'musical',
   'bounce',
   'and',
   'warm',
   'seas',
   'lapping',
   'on',
   'island',
   'shores',
   '.',
   'and',
   'just',
   'enough',
   'science',
   'to',
   'send',
   'you',
   'home',
   'thinking',
   '.'],
  'subj'),
 (['it',
   'is',
   'not',
   'a',
   'mass-market',
   'entertainment',
   'but',
   'an',
   'uncompromising',
   'attempt',
   'by',
   'one',
   'artist',
   'to',
   'think',
   'about',
   'another',
   '.'],
  'subj')]

In [30]:
obj_docs[0:3]

[(['the',
   'movie',
   'begins',
   'in',
   'the',
   'past',
   'where',
   'a',
   'young',
   'boy',
   'named',
   'sam',
   'attempts',
   'to',
   'save',
   'celebi',
   'from',
   'a',
   'hunter',
   '.'],
  'obj'),
 (['emerging',
   'from',
   'the',
   'human',
   'psyche',
   'and',
   'showing',
   'characteristics',
   'of',
   'abstract',
   'expressionism',
   ',',
   'minimalism',
   'and',
   'russian',
   'constructivism',
   ',',
   'graffiti',
   'removal',
   'has',
   'secured',
   'its',
   'place',
   'in',
   'the',
   'history',
   'of',
   'modern',
   'art',
   'while',
   'being',
   'created',
   'by',
   'artists',
   'who',
   'are',
   'unconscious',
   'of',
   'their',
   'artistic',
   'achievements',
   '.'],
  'obj'),
 (['spurning',
   'her',
   "mother's",
   'insistence',
   'that',
   'she',
   'get',
   'on',
   'with',
   'her',
   'life',
   ',',
   'mary',
   'is',
   'thrown',
   'out',
   'of',
   'the',
   'house',
   ',',
   'rejecte

## Train_test_split

> We separately split subjective and objective instances to keep a balanced uniform class distribution in both train and test sets.

In [32]:
train_subj_docs = subj_docs[:80]
test_subj_docs = subj_docs[80:100]
train_obj_docs = obj_docs[:80]
test_obj_docs = obj_docs[80:100]
training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs

## Normalization

In [34]:
sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])

In [38]:
all_words_neg[:10]

['smart',
 'and',
 'alert',
 ',',
 'thirteen',
 'conversations',
 'about',
 'one',
 'thing',
 'is']

> We use simple unigram word features, handling negation:

In [42]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
print(f'Length of unigram feats: {len(unigram_feats)}')
print(unigram_feats)

Length of unigram feats: 83
['.', 'the', ',', 'a', 'and', 'of', 'to', 'is', 'in', 'with', 'it', 'that', 'his', 'on', 'for', 'an', 'who', 'by', 'he', 'from', 'her', '"', 'film', 'as', 'this', 'movie', 'their', 'but', 'one', 'at', 'about', 'the_NEG', 'a_NEG', 'to_NEG', 'are', "there's", '(', 'story', 'when', 'so', 'be', ',_NEG', ')', 'they', 'you', 'not', 'have', 'like', 'will', 'all', 'into', 'out', 'she', 'what', 'life', 'has', 'its', 'only', 'more', 'even', '--', ':', 'can', ';', 'home', 'look', "it's", 'if', 'where', 'most', 'him', 'search', 'but_NEG', 'love', 'both', 'make', 'begins', 'some', 'two', 'of_NEG', 'made', 'which', 'them']


In [40]:
sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

> We apply features to obtain a feature-value representation of our datasets

## Training and results

In [43]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)

> We can now train our classifier on the training set, and subsequently output the evaluation results:

In [45]:
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)
                                       
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))


Training classifier
Evaluating NaiveBayesClassifier results...
Accuracy: 0.8
F-measure [obj]: 0.8
F-measure [subj]: 0.8
Precision [obj]: 0.8
Precision [subj]: 0.8
Recall [obj]: 0.8
Recall [subj]: 0.8


In [47]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [48]:
sentences = ["VADER is smart, handsome, and funny.", # positive sentence example
    "VADER is smart, handsome, and funny!", # punctuation emphasis handled correctly (sentiment intensity adjusted)
    "VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
    "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
    "VADER is VERY SMART, handsome, and FUNNY!!!",# combination of signals - VADER appropriately adjusts intensity
    "VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!",# booster words & punctuation make this close to ceiling for score
    "The book was good.",         # positive sentence
    "The book was kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
    "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
    "A really bad, horrible book.",       # negative sentence with booster words
    "At least it isn't a horrible book.", # negated negative sentence with contraction
    ":) and :D",     # emoticons handled
    "",              # an empty string is correctly handled
    "Today sux",     #  negative slang handled
    "Today sux!",    #  negative slang with punctuation emphasis handled
    "Today SUX!",    #  negative slang with capitalization emphasis
    "Today kinda sux! But I'll get by, lol" # mixed sentiment example with slang and constrastive conjunction "but"
]
paragraph = "It was one of the worst movies I've seen, despite good reviews. \
    Unbelievably bad acting!! Poor direction. VERY poor production. \
    The movie was bad. Very bad movie. VERY bad movie. VERY BAD movie. VERY BAD movie!"

In [51]:
from nltk import tokenize
lines_list = tokenize.sent_tokenize(paragraph)
sentences.extend(lines_list)

In [55]:
tricky_sentences = [
   "Most automated sentiment analysis tools are shit.",
   "VADER sentiment analysis is the shit.",
   "Sentiment analysis has never been good.",
   "Sentiment analysis with VADER has never been this good.",
   "Warren Beatty has never been so entertaining.",
   "I won't say that the movie is astounding and I wouldn't claim that \
   the movie is too banal either.",
   "I like to hate Michael Bay films, but I couldn't fault this one",
   "I like to hate Michael Bay films, BUT I couldn't help but fault this one",
   "It's one thing to watch an Uwe Boll film, but another thing entirely \
   to pay for it",
   "The movie was too good",
   "This movie was actually neither that funny, nor super witty.",
   "This movie doesn't care about cleverness, wit or any other kind of \
   intelligent humor.",
   "Those who find ugly meanings in beautiful things are corrupt without \
   being charming.",
   "There are slow and repetitive parts, BUT it has just enough spice to \
   keep it interesting.",
   "The script is not fantastic, but the acting is decent and the cinematography \
   is EXCELLENT!",
   "Roger Dodger is one of the most compelling variations on this theme.",
   "Roger Dodger is one of the least compelling variations on this theme.",
   "Roger Dodger is at least compelling as a variation on the theme.",
   "they fall in love with the product",
   "but then it breaks",
   "usually around the time the 90 day warranty expires",
   "the twin towers collapsed today",
   "However, Mr. Carter solemnly argues, his client carried out the kidnapping \
   under orders and in the ''least offensive way possible.''"
]

sentences.extend(tricky_sentences)
for sentence in sentences:
    sid = SentimentIntensityAnalyzer()
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
    print('\n')

VADER is smart, handsome, and funny.
compound: 0.8316, neg: 0.0, neu: 0.254, pos: 0.746, 

VADER is smart, handsome, and funny!
compound: 0.8439, neg: 0.0, neu: 0.248, pos: 0.752, 

VADER is very smart, handsome, and funny.
compound: 0.8545, neg: 0.0, neu: 0.299, pos: 0.701, 

VADER is VERY SMART, handsome, and FUNNY.
compound: 0.9227, neg: 0.0, neu: 0.246, pos: 0.754, 

VADER is VERY SMART, handsome, and FUNNY!!!
compound: 0.9342, neg: 0.0, neu: 0.233, pos: 0.767, 

VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!
compound: 0.9469, neg: 0.0, neu: 0.294, pos: 0.706, 

The book was good.
compound: 0.4404, neg: 0.0, neu: 0.508, pos: 0.492, 

The book was kind of good.
compound: 0.3832, neg: 0.0, neu: 0.657, pos: 0.343, 

The plot was good, but the characters are uncompelling and the dialog is not great.
compound: -0.7042, neg: 0.327, neu: 0.579, pos: 0.094, 

A really bad, horrible book.
compound: -0.8211, neg: 0.791, neu: 0.209, pos: 0.0, 

At least it isn't a horrible book

# NLP - French POC